In [2]:
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import csv

# input ncbi accession number for the genome of interest
id = 'NC_001416.1'

# input paths to gff and fasta files for the genome of interest
path_to_gff = 'gff/'
path_to_fasta = 'fasta/'

In [3]:
# helper for get_genes
def get_product(str, to_find):
    pos1 = str.find(to_find)
    pos2 = str.find(';', pos1, len(str))
    return str[pos1 + len(to_find):pos2]

# opens gff3 file, uses it to access every gene's id, start position, end position, name, and strand (in that order)
def get_genes(path_id):
    f = open(path_id + '.gff3', 'r')
    genes = []
    for line in f.readlines():
        l = line.split('\t')
        if len(l) > 4 and l[2] == 'gene':
            genes.append((get_product(l[8], 'ID=gene-'), int(l[3]), int(l[4]), get_product(l[8], 'Name='), l[6]))
        elif len(l) > 4 and id == 'EF056009.1' and l[2] == 'CDS':
            genes.append((get_product(l[8], 'product='), int(l[3]), int(l[4]), get_product(l[8], 'Name='), l[6]))
    return genes

# opens fasta file, accesses seqs of all of the genes + strand directions
def get_seqs(s1e2, path_id):
    seq = SeqIO.read(path_id + '.fasta', 'fasta')
    gene = []
    for i in range(len(s1e2)):
        if s1e2[i][4] == "-":
#            print('negative')
            gene.append(str((seq.seq[s1e2[i][1] : s1e2[i][2]]).reverse_complement()))
        elif s1e2[i][4] == "+":
            gene.append(str(seq.seq[s1e2[i][1] : s1e2[i][2]]))
#            print('positive')
        else:
            print('strand direction not identifiable:')
            print(s1e2[i][4])
    return gene

# used to determine whether the TTTV site is viable
# current requirements: 
def passes_tests(a_str, start):
    if start >= len(a_str) - 28:
        return False
    t = extract_primer(a_str, start)
    if 'GAAGAC' in t or 'GTCTTC' in t:
        return False
    return True

# for a given gene sequence, find as many entries of sites_to_look_for
# as possible, and compile them into a list of potential primer sites
def find_primer_candidates(gene_seq, sites_to_look_for):
    start = 0
    while True:
        min_start = 999999999
        min_idx = -1
        for i in range(len(sites_to_look_for)):
            possible_start = gene_seq.find(sites_to_look_for[i], start)
            if possible_start > -1:
                min_start = min(possible_start, min_start)
                min_idx = i
        if min_start == 999999999: return
        start = min_start
        if passes_tests(gene_seq, start):
            yield start
        start += (len(sites_to_look_for[min_idx]) + 28)
        
# given the gene sequence, extract the 28bp primer immediately
# downstream of the identified primer location
def extract_primer(seq, tttv_loc):
    return seq[tttv_loc+4:tttv_loc+32]

# given a set of candidate primer locations (cand), the length of the 
# corresponding gene (length), and two ideal %downstreams of the gene 
# in question (loc1 and loc2), find the two primer candidates that 
# are closest to loc1 and loc2 in terms of how far downstream in the gene
# they are
def get_indices(cand, loc1, loc2, length):
    int1 = int(loc1 * length)
    int2 = int(loc2 * length)
    
    i = 0
    not_found = True
    while not_found:
        for candidate in cand:
            if candidate in range(int1 - i, int1 + i):
                sol1 = candidate
                not_found = False
        i += 1
        
    not_found = True
    i = 0
    while not_found:
        for candidate in cand:
            if candidate in range(int2 - i, int2 + i) and candidate is not sol1:
                sol2 = candidate
                not_found = False
        i += 1
    
    return [sol1, sol2]
    

In [5]:
gene_name_start_end_locs = get_genes(path_to_gff + id)
genes = get_seqs(gene_name_start_end_locs, path_to_fasta + id)

In [6]:
# this is more of a sanity check than anything -- not super important in and of itself
t = list(find_primer_candidates(genes[0], ["TTTC", "TTTG", "TTTA"]))
print(t)
print(len(genes[0]))

[31, 117, 343, 416]
545


In [22]:
# calculate the primer candidates for each set, as well as how many there are for each gene in question
primer_candidates = []
for i in range(len(genes)):
    primer_candidates.append(list(find_primer_candidates(genes[i],["TTTC", "TTTG", "TTTA"])))
count = [len(primer_start_pos_list) for primer_start_pos_list in primer_candidates]
print(count)

[4, 16, 1, 18, 6, 2, 2, 5, 1, 4, 4, 5, 5, 5, 5, 14, 3, 7, 5, 6, 23, 187, 186, 186, 185, 4, 3, 11, 9, 4, 18, 0, 14, 25, 71, 14, 10, 1, 0, 1, 2, 1, 2, 2, 40, 7, 6, 4, 23, 22, 1, 1, 18, 13, 1, 6, 7, 38, 31, 7, 7, 14, 8, 16, 3, 2, 2, 1, 0, 1, 5, 5, 2, 4, 9, 0, 0, 1, 2, 0, 6, 3, 0, 1, 3, 3, 3, 1, 0, 4, 7, 1]


In [28]:
# for each gene, first determine how many primer candidates there are.
# if 0: indicate to the user that this gene has no primer candidates. Can't do much else here.
# if 1: create a fwd and rev primer out of that primer candidate (called <gene name>_1_FWD and <gene name>_1_REV)
#       and add it to the list of primers
# if 2: find the best two candidates with get_indices(); create fwd and rev primers out of these two candidates
primers = []
for i in range(len(genes)):
    if count[i] > 1:
        indices = get_indices(primer_candidates[i], 0.2, 0.5, len(genes[i]))
        prime1 = Seq(extract_primer(genes[i],indices[0]))
        primer1_fwd = 'AGAT' + str(prime1).lower() + 'G'
        primer1_rev = 'GAAAC' + str(prime1.reverse_complement()).lower()
        prime2 = Seq(extract_primer(genes[i],indices[1]))
        primer2_fwd = 'AGAT' + str(prime2).lower() + 'G'
        primer2_rev = 'GAAAC' + str(prime2.reverse_complement()).lower()
        primers.append([gene_name_start_end_locs[i][0] + '_1_FWD', primer1_fwd, gene_name_start_end_locs[i][0] + '_1_REV', primer1_rev, round(indices[0] / len(genes[i]), 2)])
        primers.append([gene_name_start_end_locs[i][0] + '_2_FWD', primer2_fwd, gene_name_start_end_locs[i][0] + '_2_REV', primer2_rev, round(indices[1] / len(genes[i]), 2)])
    elif count[i] == 1:
        prime1 = Seq(extract_primer(genes[i],primer_candidates[i][0]))
        primer1_fwd = 'AGAT' + str(prime1).lower() + 'G'
        primer1_rev = 'GAAAC' + str(prime1.reverse_complement()).lower()
        primers.append([gene_name_start_end_locs[i][0] + '_1_FWD', primer1_fwd,gene_name_start_end_locs[i][0] + '_1_REV', primer1_rev, round(primer_candidates[i][0] / len(genes[i]),2)])
    else:
        print('gene ' + str(i) + ', ' + gene_name_start_end_locs[i][0] + ', had no possible primer candidate sequences.')
        primers.append([])

gene 31, lambdap30, had no possible primer candidate sequences.
gene 38, lambdap35, had no possible primer candidate sequences.
gene 68, lambdap59, had no possible primer candidate sequences.
gene 75, lambdap65, had no possible primer candidate sequences.
gene 76, lambdap66, had no possible primer candidate sequences.
gene 79, lambdap69, had no possible primer candidate sequences.
gene 82, lambdap72, had no possible primer candidate sequences.
gene 88, lambdap91, had no possible primer candidate sequences.


In [29]:
with open(id + '_primers' + '.csv','w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerows(primers)

In [27]:
round(0.102147120938,3)

0.102